# Section 1
according to the lectures, we define the privacy as the action of removing somethign from the db without affecting the statistics of the query.

## Generate parallel databases

In [1]:
import torch

# the number of entries in our database
num_entries = 5000

db = torch.rand(num_entries) > 0.5
db

tensor([0, 0, 1,  ..., 1, 1, 1], dtype=torch.uint8)

In [3]:
db[0:5]

tensor([0, 0, 1, 0, 1], dtype=torch.uint8)

In [19]:
# Create a funcion to remove element x in db.
def get_parallel_db(db, remove_index):
    return torch.cat((db[0:remove_index], db[remove_index+1:]))

In [20]:
get_parallel_db(db, 3)

tensor([0, 0, 1,  ..., 1, 1, 1], dtype=torch.uint8)

In [23]:
# We are now removing x + 1 elemetns of the db and creating a db with at least all
# all the participants removed.

def get_parallel_dbs(db):
    
    parallel_dbs = []
    
    for i in range(len(db)):
        pdb = get_parallel_db(db,i)
        parallel_dbs.append(pdb)
    return parallel_dbs
            

In [28]:
pdbs = get_parallel_dbs(db)


In [26]:
#  Final function thath condeces all the work.

def create_db_and_parallel(num_entries):

    db = torch.rand(num_entries) > 0.5
    pdbs = get_parallel_dbs(db)
    return db, pdbs

db, pdbs = create_db_and_parallel(30)
    

In [27]:
db

tensor([1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0], dtype=torch.uint8)

In [29]:
db.shape

torch.Size([30])

In [31]:
len(pdbs)

30